In [ ]:
import sqlite3
from datetime import date
from docstore.extract.input import HtmlReader, OpenAIExtractor

html_read = HtmlReader()
openai_extract = OpenAIExtractor(model="gpt-4.1-nano", api_key_file="/home/.openai/key")

In [ ]:
# Step 1: Connect to SQLite DB (will create file if it doesn't exist)
conn = sqlite3.connect("/home/jovyan/data/ecoi.db")
cursor = conn.cursor()

# Step 2: Create the `pages` table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS pages (
        id INTEGER PRIMARY KEY,
        exist BOOLEAN,
        source TEXT,
        output TEXT,
        requested TEXT
    )
""")

# Index on page_exists (formerly "exists")
cursor.execute("""
    CREATE INDEX IF NOT EXISTS idx_pages_page_exists
    ON pages (exist)
    """
)

# Index on requested
cursor.execute("""
    CREATE INDEX IF NOT EXISTS idx_pages_requested
    ON pages (requested)
    """
)

In [ ]:
id = 2125418

In [ ]:
n_outputs = 0
while id > 1_000_000:

    print(f"id={id}. n_outputs={n_outputs}", end='\r', flush=True)

    output = html_read.get_html(id, extract_class="col-md-3")
    if output is not None:
        n_outputs += 1
        parsed = {
            "id": id,
            "exist": True,
            "source": html_read.get_page(id),
            "output": output,
            "requested": str(date.today())
        }
    else:
        parsed = {
            "id": id,
            "exist": False,
            "source": None,
            "output": None,
            "requested": str(date.today())
        }
    id -= 1

    cursor.execute("""
        INSERT OR REPLACE INTO pages (id, exist, source, output, requested)
        VALUES (?, ?, ?, ?, ?)
        """,
        (parsed["id"], parsed["exist"], parsed["source"], str(parsed["output"]), parsed["requested"])
    )

    conn.commit()

conn.close()